In [1]:
%%capture
!git clone https://github.com/nyu-mll/jiant.git
%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./
!pip install networkx==1.11

In [2]:
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache
import jiant.proj.main.export_model as export_model
import jiant.proj.main.scripts.configurator as configurator
import jiant.proj.main.runscript as main_runscript
import jiant.shared.caching as caching
import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os
import sys
from google.colab import drive
# Adjust File path/partitions
sys.path.insert(0, "/content/jiant")
drive.mount('/content/gdrive')

In [3]:
%%capture
%cd /content
# Download the tasks to be used
!PYTHONPATH=/content/jiant python jiant/jiant/scripts/download_data/runscript.py \
    download \
    --tasks pawsx \
    --output_path=/content/tasks/

In [6]:
export_model.export_model(
    hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
    output_base_path="./models/bert-base-multilingual-cased",
)

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and cache each task
languages = ['en', 'de', 'es', 'fr', 'ja', 'ko', 'zh']
task_names = ["pawsx_"+lan for lan in languages]
for task_name in task_names:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"./tasks/configs/{task_name}_config.json",
        hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
        output_dir=f"./cache/{task_name}",
        phases=["val"],
    ))

In [ ]:
# Tokenize and cache each task
languages = ["af","ar","bg","de","el","en","et","eu","fa","fi","fr","he","hi","hu","id","it","ja","kk","ko","mr","nl","pt","ru","ta",
             "te","th","tl","tr","ur","vi","yo","zh"]
#task_names = ["udpos_{}".format(lang) for lang in languages]
task_names = ["udpos_en"]
for task_name in task_names:
  try:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"./tasks/configs/{task_name}_config.json",
        hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
        output_dir=f"./cache/{task_name}",
        phases=["train","val"],
    ))
  except:
    print(task_name)


UdposTask
  [train]: /content/tasks/data/udpos_en/train-en.tsv
  [val]: /content/tasks/data/udpos_en/val-en.tsv
  [test]: /content/tasks/data/udpos_en/test-en.tsv


In [ ]:
# Build the config. file needed for training with Jiant
jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="./tasks/configs",
    task_cache_base_path="./cache",
    val_task_name_list=task_names,
    train_batch_size=8,
    eval_batch_size=8,
    epochs=3,
    num_gpus=1,
).create_config()
os.makedirs("./run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, "./run_configs/jiant_run_config.json")
display.show_json(jiant_run_config)

In [10]:
# Start Training/Evaluating the model on the chosen tasks
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/jiant_run_config.json",
    output_dir="./runs/run_udpos",
    hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
    model_path="/content/gdrive/MyDrive/thesis/multi_copa-multirc-paws/run1/best_model.p",
    model_config_path="./models/bert-base-multilingual-cased/model/config.json",
    learning_rate=2e-5,
    eval_every_steps=500,
    
    do_train=False,
    do_val=True,
    do_save = True,
    force_overwrite=False
)

main_runscript.run_loop(run_args)

  jiant_task_container_config_path: ./run_configs/jiant_run_config.json
  output_dir: ./runs/run_udpos
  hf_pretrained_model_name_or_path: bert-base-multilingual-cased
  model_path: /content/gdrive/MyDrive/thesis/multi_copa-multirc-paws/run1/best_model.p
  model_config_path: ./models/bert-base-multilingual-cased/model/config.json
  model_load_mode: from_transformers
  do_train: False
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 500
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: False
  seed: -1
  learning_rate: 2e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
Using seed: 937593697
{
  "jiant_task_co

/content/jiant/jiant/proj/main/modeling/model_setup.py:174: UserWarning: The following weights were not loaded: dict_keys(['encoder.embeddings.position_ids', 'encoder.embeddings.word_embeddings.weight', 'encoder.embeddings.position_embeddings.weight', 'encoder.embeddings.token_type_embeddings.weight', 'encoder.embeddings.LayerNorm.weight', 'encoder.embeddings.LayerNorm.bias', 'encoder.encoder.layer.0.attention.self.query.weight', 'encoder.encoder.layer.0.attention.self.query.bias', 'encoder.encoder.layer.0.attention.self.key.weight', 'encoder.encoder.layer.0.attention.self.key.bias', 'encoder.encoder.layer.0.attention.self.value.weight', 'encoder.encoder.layer.0.attention.self.value.bias', 'encoder.encoder.layer.0.attention.output.dense.weight', 'encoder.encoder.layer.0.attention.output.dense.bias', 'encoder.encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.encoder.layer.0.intermediate.dense.weight', 'encoder.encoder

No optimizer decay for:
  encoder.embeddings.LayerNorm.weight
  encoder.embeddings.LayerNorm.bias
  encoder.encoder.layer.0.attention.self.query.bias
  encoder.encoder.layer.0.attention.self.key.bias
  encoder.encoder.layer.0.attention.self.value.bias
  encoder.encoder.layer.0.attention.output.dense.bias
  encoder.encoder.layer.0.attention.output.LayerNorm.weight
  encoder.encoder.layer.0.attention.output.LayerNorm.bias
  encoder.encoder.layer.0.intermediate.dense.bias
  encoder.encoder.layer.0.output.dense.bias
  encoder.encoder.layer.0.output.LayerNorm.weight
  encoder.encoder.layer.0.output.LayerNorm.bias
  encoder.encoder.layer.1.attention.self.query.bias
  encoder.encoder.layer.1.attention.self.key.bias
  encoder.encoder.layer.1.attention.self.value.bias
  encoder.encoder.layer.1.attention.output.dense.bias
  encoder.encoder.layer.1.attention.output.LayerNorm.weight
  encoder.encoder.layer.1.attention.output.LayerNorm.bias
  encoder.encoder.layer.1.intermediate.dense.bias
  encode


{
  "aggregated": 0.46621428571428575,
  "pawsx_en": {
    "loss": 0.7324293696880341,
    "metrics": {
      "major": 0.445,
      "minor": {
        "acc": 0.445
      }
    }
  },
  "pawsx_de": {
    "loss": 0.6991619908809662,
    "metrics": {
      "major": 0.4835,
      "minor": {
        "acc": 0.4835
      }
    }
  },
  "pawsx_es": {
    "loss": 0.6888409175872803,
    "metrics": {
      "major": 0.546,
      "minor": {
        "acc": 0.546
      }
    }
  },
  "pawsx_fr": {
    "loss": 0.7031103682518005,
    "metrics": {
      "major": 0.43,
      "minor": {
        "acc": 0.43
      }
    }
  },
  "pawsx_ja": {
    "loss": 0.693663871049881,
    "metrics": {
      "major": 0.4825,
      "minor": {
        "acc": 0.4825
      }
    }
  },
  "pawsx_ko": {
    "loss": 0.7015168886184693,
    "metrics": {
      "major": 0.442,
      "minor": {
        "acc": 0.442
      }
    }
  },
  "pawsx_zh": {
    "loss": 0.7049945902824402,
    "metrics": {
      "major": 0.4345,
      "

In [ ]:
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/jiant_run_config.json",
    output_dir="./runs/run1",
    hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
    model_path="./models/bert-base-multilingual-cased/model/model.p",
    model_config_path="./models/bert-base-multilingual-cased/model/config.json",
    learning_rate=2e-5,
    eval_every_steps=500,
    
    do_train=True,
    do_val=True,
    do_save = True,
    force_overwrite=True,
)

main_runscript.run_loop(run_args)

In [ ]:
!cp -r '/content/runs' '/content/gdrive/MyDrive/thesis/udpos_multitrained'

In [ ]:
!cp -r '/content/run_configs' '/content/gdrive/MyDrive/thesis/multi-trained/config'

Finally, we should see the validation scores for all three tasks. We are not winning any awards with these scores, but this example should show how easy it is to run multi-task training in `jiant`.